# Topic modeling on continental data (Africa) using LdaModel from gensim

## Install pyLDAvis

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=4b543804274b8da461c891bca29452e75b365b271b283f9fa449a0b4562595b9
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


## Import Libraries

In [ ]:
import pandas as pd
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain
import re

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]
/usr/local/lib/python3.7/dist-packages/scipy/io/matlab/mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Data

In [ ]:
twitter_data = pd.read_csv('/content/drive/MyDrive/NLP/Datasets/Twitter/Continent & Country Tweets 2022-04-05.csv')
twitter_data

,screen_name,Source,tweets,time_stamp
0,SaraTigray8,Africa,RT @GaroweOnline: Ethiopia's #Tigray war: Insi...,2022-04-05 06:08:30+00:00
1,NSTED_Africa,Africa,RT @zimadventures: 📍Allow us to announce anoth...,2022-04-05 06:08:29+00:00
2,blkbrrie,Africa,"RT @equalityAlec: To put this into context, D....",2022-04-05 06:08:29+00:00
3,the_davidatta,Africa,"RT @TrojanAid: 📢TODAY'S THOUGHT\n\n""The so-cal...",2022-04-05 06:08:28+00:00
4,Official21Erica,Africa,"RT @bra_louis: With the QR Code, Ghana has one...",2022-04-05 06:08:28+00:00
...,...,...,...,...
356,1homiey,South Africa,RT @Manqoba_Mbuli: South Africa here is the Fa...,2022-04-05 06:03:39+00:00
357,WildEye_News,South Africa,"[South Africa] 23-year-old poacher, Rich Ricar...",2022-04-05 06:03:38+00:00
358,KatesCurious,South Africa,Wow. I feel like over above this guy in the tr...,2022-04-05 06:03:35+00:00
359,crimemind3,South Africa,"RT @Ayandi07684809: Malema "" Land and Jobs nam...",2022-04-05 06:03:35+00:00


## Clean Data

In [ ]:
## Rename tweets column to tweet
twitter_data.rename(columns = {'tweets':'tweet'}, inplace = True)

In [ ]:
def text_cleaner (text):
  text = re.sub(r'@[A-Za-z0-9]+','',text) ## remove @ mentions
  text = re.sub(r'#','',text) ## remove # symbol
  text = re.sub(r'^RT+','',text) ## remove RT
  text = re.sub(r'https?:\/\/\S+','',text) ## remove hyperlink
  text = re.sub(r'[^\w\s]','',text) ## remove everything apart from words and space
  text = re.sub(r'_',' ',text) ## remove underscore
  text = re.sub(r'\n',' ',text) ## remove \n

  return text

In [ ]:
## Create clean text column

twitter_data['cleaned_tweet'] = twitter_data['tweet'].apply(text_cleaner)
twitter_data = twitter_data[['screen_name','Source','tweet','cleaned_tweet','time_stamp']]
twitter_data

,screen_name,Source,tweet,cleaned_tweet,time_stamp
0,SaraTigray8,Africa,RT @GaroweOnline: Ethiopia's #Tigray war: Insi...,Ethiopias Tigray war Inside Mekelle cut off ...,2022-04-05 06:08:30+00:00
1,NSTED_Africa,Africa,RT @zimadventures: 📍Allow us to announce anoth...,Allow us to announce another weekend Trip fo...,2022-04-05 06:08:29+00:00
2,blkbrrie,Africa,"RT @equalityAlec: To put this into context, D....",To put this into context DC is already the m...,2022-04-05 06:08:29+00:00
3,the_davidatta,Africa,"RT @TrojanAid: 📢TODAY'S THOUGHT\n\n""The so-cal...",TODAYS THOUGHT The socalled international l...,2022-04-05 06:08:28+00:00
4,Official21Erica,Africa,"RT @bra_louis: With the QR Code, Ghana has one...",louis With the QR Code Ghana has one of the ...,2022-04-05 06:08:28+00:00
...,...,...,...,...,...
356,1homiey,South Africa,RT @Manqoba_Mbuli: South Africa here is the Fa...,Mbuli South Africa here is the Fastest under...,2022-04-05 06:03:39+00:00
357,WildEye_News,South Africa,"[South Africa] 23-year-old poacher, Rich Ricar...",South Africa 23yearold poacher Rich Ricardo Ch...,2022-04-05 06:03:38+00:00
358,KatesCurious,South Africa,Wow. I feel like over above this guy in the tr...,Wow I feel like over above this guy in the tro...,2022-04-05 06:03:35+00:00
359,crimemind3,South Africa,"RT @Ayandi07684809: Malema "" Land and Jobs nam...",Malema Land and Jobs namhlanje after elect...,2022-04-05 06:03:35+00:00


In [ ]:
## Define Filters
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [ ]:
## Define function

def text_cleaner2 (text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [ ]:
twitter_data['cleaned_tweet2'] = twitter_data['cleaned_tweet'].apply(text_cleaner2)
twitter_data = twitter_data[['screen_name','Source','tweet','cleaned_tweet','cleaned_tweet2','time_stamp']]
twitter_data

,screen_name,Source,tweet,cleaned_tweet,cleaned_tweet2,time_stamp
0,SaraTigray8,Africa,RT @GaroweOnline: Ethiopia's #Tigray war: Insi...,Ethiopias Tigray war Inside Mekelle cut off ...,"[ethiopia, tigray, war, inside, mekelle, cut, ...",2022-04-05 06:08:30+00:00
1,NSTED_Africa,Africa,RT @zimadventures: 📍Allow us to announce anoth...,Allow us to announce another weekend Trip fo...,"[allow, u, announce, another, weekend, trip, y...",2022-04-05 06:08:29+00:00
2,blkbrrie,Africa,"RT @equalityAlec: To put this into context, D....",To put this into context DC is already the m...,"[put, context, dc, already, policed, city, soc...",2022-04-05 06:08:29+00:00
3,the_davidatta,Africa,"RT @TrojanAid: 📢TODAY'S THOUGHT\n\n""The so-cal...",TODAYS THOUGHT The socalled international l...,"[today, thought, socalled, international, law,...",2022-04-05 06:08:28+00:00
4,Official21Erica,Africa,"RT @bra_louis: With the QR Code, Ghana has one...",louis With the QR Code Ghana has one of the ...,"[louis, qr, code, ghana, one, advanced, inclus...",2022-04-05 06:08:28+00:00
...,...,...,...,...,...,...
356,1homiey,South Africa,RT @Manqoba_Mbuli: South Africa here is the Fa...,Mbuli South Africa here is the Fastest under...,"[mbuli, south, africa, fastest, under18, femal...",2022-04-05 06:03:39+00:00
357,WildEye_News,South Africa,"[South Africa] 23-year-old poacher, Rich Ricar...",South Africa 23yearold poacher Rich Ricardo Ch...,"[south, africa, 23yearold, poacher, rich, rica...",2022-04-05 06:03:38+00:00
358,KatesCurious,South Africa,Wow. I feel like over above this guy in the tr...,Wow I feel like over above this guy in the tro...,"[wow, feel, like, guy, trollys, bravery, reckl...",2022-04-05 06:03:35+00:00
359,crimemind3,South Africa,"RT @Ayandi07684809: Malema "" Land and Jobs nam...",Malema Land and Jobs namhlanje after elect...,"[malema, land, job, namhlanje, electionsmalema...",2022-04-05 06:03:35+00:00


## Create Dictionary from the articles

In [ ]:
#create dictionary
dictionary = corpora.Dictionary(twitter_data['cleaned_tweet2'])
#Total number of non-zeroes in the BOW matrix (sum of the number of unique words per document over the entire corpus).
print(dictionary.num_nnz)

3741


## Create document term matrix

In [ ]:
#create document term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in twitter_data['cleaned_tweet2'] ]
print(len(doc_term_matrix))

361


## Instantiate LDA model

In [ ]:
lda = gensim.models.ldamodel.LdaModel

## Fit LDA model on the dataset

In [ ]:
num_topics=2
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)

CPU times: user 6.66 s, sys: 115 ms, total: 6.77 s
Wall time: 6.65 s


## Print the topics identified by LDA model

In [ ]:
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.021*"africa" + 0.015*"south" + 0.012*"kenya" + 0.009*"mauritius" + 0.006*"fta" + 0.005*"put" + 0.005*"time" + 0.004*"world" + 0.004*"biggest" + 0.004*"asked"'),
 (1,
  '0.029*"africa" + 0.018*"south" + 0.010*"kenya" + 0.007*"world" + 0.007*"mauritius" + 0.006*"year" + 0.006*"amp" + 0.005*"national" + 0.004*"old" + 0.003*"fuel"')]

## Visualize the LDA model results

In [ ]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

## Find which articles were marked in which cluster

In [ ]:
# Assigns the topics to the documents in corpus
lda_corpus = ldamodel[doc_term_matrix]

In [ ]:
 all_topics = ldamodel.get_document_topics(lda_corpus)
 all_topics_csr = gensim.matutils.corpus2csc(all_topics)
 all_topics_numpy = all_topics_csr.T.toarray()
 all_topics_df = pd.DataFrame(all_topics_numpy)

In [ ]:
twitter_data['topic'] = all_topics_df.columns.get_indexer(all_topics_df.apply('idxmax', axis=1))


In [ ]:
twitter_data['topic'].value_counts(normalize=True)

0    1.0
Name: topic, dtype: float64